# IBM Watson OpenScale Lab instructions

This notebook should be run in a Watson Studio project, using with **Python 3.6 with Spark** runtime environment. **If you are viewing this in Watson Studio and do not see Python 3.6 with Spark in the upper right corner of your screen, please update the runtime now.** It requires service credentials for the following Cloud services:
  * IBM Watson OpenScale
  * Watson Machine Learning
  
If you have a paid Cloud account, you may also provision a **Databases for PostgreSQL** or **Db2 Warehouse** service to take full advantage of integration with Watson Studio and continuous learning services. If you choose not to provision this paid service, you can use the free internal PostgreSQL storage with OpenScale, but will not be able to configure continuous learning for your model.

The notebook will train, create and deploy a Propensity to Buy Model, configure OpenScale to monitor that deployment, and inject seven days' worth of historical records and measurements for viewing in the OpenScale Insights dashboard.

# Business Scenario

National Chemical Company (NCC) operates two plants plants in South Lousiana.  Both plants produce the same chemical.  Both plants use the same suppliers.  Both plants are managed by the same people.  The only difference between the two plants is age.  Plant A is brand new.  It opened less than a year ago.  Plant B is more than 20 years old.

Bad Yields cost NCC millions of dollars each year.  A bad yield just means that the final product produced by the factory does not meet quality standards.  Ideally, they would like to predict a bad yield a few days in advance, drill down to see why there is a prediction of a bad yield and make corrections.  To accomplish this, NCC turned to IBM and Watson.  

In this notebook, you will construct a machine learning model and deploy that model to a Watson Machine Learning Service.  This model is based on historical data and predicts the probability of a bad yield.  

After the model is built and deployed, the notebook configures Watson Openscale.  Watson Openscale will allow NCC to monitor the accuracy of the model over-time and understand the key factors that cause  bad yields.  Knowing why a bad yield will occur allows the plant operators to make adjustments and prevent them from occuring.

Another key issue that NCC is worried about is the bad sensor readings in Plant B, the older plant.  They are concerned that the bad sensor readings will lead to invalid predictions.  Given that Plant A and B an should have a similar percentage of predicted Bad Yield yields, we will use the bias and fairness detection monitors inside Watson Openscale to correct bias that may occur because of the older sensors in Plant B.






# Package installation

In [ ]:
!rm -rf $PIP_BUILD
!pip install psycopg2-binary | tail -n 1
!pip install --upgrade watson-machine-learning-client --no-cache | tail -n 1
!pip install --upgrade ibm-ai-openscale --no-cache | tail -n 1
!pip install --upgrade numpy --no-cache | tail -n 1
!pip install --upgrade lime --no-cache | tail -n 1
!pip install --upgrade SciPy --no-cache | tail -n 1

#### Provision services and configure credentials

If you have not already, provision an instance of IBM Watson OpenScale using the [OpenScale link in the Cloud catalog](https://cloud.ibm.com/catalog/services/ai-openscale).

Your Cloud API key can be generated by going to the [**Users** section of the Cloud console](https://cloud.ibm.com/iam#/users). From that page, click your name, scroll down to the **API Keys** section, and click **Create an IBM Cloud API key**. Give your key a name and click **Create**, then copy the created key and paste it below.

In [ ]:
CLOUD_API_KEY = "PFHEY2Po9KcARJqfUtnv75yvAHtawFzURnMWorJedhK7"

Next you will need credentials for Watson Machine Learning. If you already have a WML instance, you may use credentials for it. To provision a new Lite instance of WML, use the [Cloud catalog](https://cloud.ibm.com/catalog/services/machine-learning), give your service a name, and click **Create**. Once your instance is created, click the **Service Credentials** link on the left side of the screen. Click the **New credential** button, give your credentials a name, and click **Add**. Your new credentials can be accessed by clicking the **View credentials** button. Copy and paste your WML credentials into the cell below.

In [ ]:
WML_CREDENTIALS ={
  "apikey": "uoUoBHKgrz-OPFGBmLNhQgy_BlHNPXCqyJYaR-5tGNGt",
  "iam_apikey_description": "Auto-generated for key 9e155d75-e70d-40c7-96ee-dba3fe9a030d",
  "iam_apikey_name": "Service credentials-1",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/97deeb0b7e78431438a00a04f20580b7::serviceid:ServiceId-755ca7bd-4d01-44b9-912d-7c05f9d81a11",
  "instance_id": "67c0aed8-ed70-4029-a923-a585e83408c4",
  "password": "b7a11f66-129c-44fc-a963-958e9e4f6452",
  "url": "https://us-south.ml.cloud.ibm.com",
  "username": "9e155d75-e70d-40c7-96ee-dba3fe9a030d"
}

This lab can use Databases for PostgreSQL, Db2 Warehouse, or a free internal verison of PostgreSQL to create a datamart for OpenScale.

If you have previously configured OpenScale, it will use your existing datamart, and not interfere with any models you are currently monitoring. Do not update the cell below.

If you do not have a paid Cloud account or would prefer not to provision this paid service, you may use the free internal PostgreSQL service with OpenScale. Do not update the cell below.

To provision a new instance of Db2 Warehouse, locate [Db2 Warehouse in the Cloud catalog](https://cloud.ibm.com/catalog/services/db2-warehouse), give your service a name, and click **Create**. Once your instance is created, click the **Service Credentials** link on the left side of the screen. Click the **New credential** button, give your credentials a name, and click **Add**. Your new credentials can be accessed by clicking the **View credentials** button. Copy and paste your Db2 Warehouse credentials into the cell below.

To provision a new instance of Databases for PostgreSQL, locate [Databases for PostgreSQL in the Cloud catalog](https://cloud.ibm.com/catalog/services/databases-for-postgresql), give your service a name, and click **Create**. Once your instance is created, click the **Service Credentials** link on the left side of the screen. Click the **New credential** button, give your credentials a name, and click **Add**. Your new credentials can be accessed by clicking the **View credentials** button. Copy and paste your Databases for PostgreSQL credentials into the cell below.

In [ ]:
DB_CREDENTIALS = None

__If you previously configured OpenScale to use the free internal version of PostgreSQL, you can switch to a new datamart using a paid database service.__ If you would like to delete the internal PostgreSQL configuration and create a new one using service credentials supplied in the cell above, set the __KEEP_MY_INTERNAL_POSTGRES__ variable below to __False__ below. In this case, the notebook will remove your existing internal PostgreSQL datamart and create a new one with the supplied credentials. __*NO DATA MIGRATION WILL OCCUR.*__

In [ ]:
KEEP_MY_INTERNAL_POSTGRES = True

# Run the notebook

At this point, the notebook is ready to run. You can either run the cells one at a time, or click the **Kernel** option above and select **Restart and Run All** to run all the cells.

# Load and explore data

## Load the training data from github

In [ ]:
!rm df_training.csv
!wget https://raw.githubusercontent.com/shadgriffin/oglabworking/master/df_training.csv

In [ ]:
from pyspark.sql import SparkSession
import pandas as pd
import json
import numpy as np

spark = SparkSession.builder.getOrCreate()
pd_data = pd.read_csv("df_training.csv", sep=",", header=0)
#df_data = spark.read.csv(path="df_training.csv", sep=",", header=True, inferSchema=True)
#df_data.head()

In [ ]:
df_training=pd_data

In [ ]:
pd_data.columns

In [ ]:
pd_data['wookie']=(np.random.randint(1, 1000,pd_data.shape[0]))/1000


In [ ]:
PLANT_B=pd_data[pd_data['PLANT_A'] ==0]
PLANT_A=pd_data[pd_data['PLANT_A']==1]
PLANT_B_0=PLANT_B[PLANT_B['BAD_YIELD']=='BAD']
PLANT_B_1=PLANT_B[PLANT_B['BAD_YIELD']=='GOOD']
#PLANT_B_1a=PLANT_B_1[PLANT_B_1['wookie']<=0.75]
#PLANT_B_1b=PLANT_B_1[PLANT_B_1['wookie']>0.75]



PLANT_B_1 = PLANT_B_1.copy()

PLANT_B_1['BAD_YIELD'] = np.where(PLANT_B_1['wookie']<=0.000001, 'BAD', 'GOOD')

In [ ]:
pd_data = PLANT_B_0.append([PLANT_B_1, PLANT_A])

pd_data=pd_data.drop(columns=['wookie'])

In [ ]:
pd.crosstab(pd_data.PLANT_A, pd_data.BAD_YIELD).apply(lambda r: r/r.sum(), axis = 1)

In [ ]:
pd_data.shape
pd_data.head()

In [ ]:
df_data = spark.createDataFrame(pd_data)
df_data.head()

## Explore data

In [ ]:
df_data.printSchema()

In [ ]:
print("Number of records: " + str(df_data.count()))

# Create a model

In [ ]:
spark_df = df_data
(train_data, test_data) = spark_df.randomSplit([0.8, 0.2], 24)

MODEL_NAME = "Yieldx Model"
DEPLOYMENT_NAME = "Yieldx Model"

print("Number of records for training: " + str(train_data.count()))
print("Number of records for evaluation: " + str(test_data.count()))

spark_df.printSchema()

In [ ]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, IndexToString, VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml import Pipeline, Model
from pyspark.ml import linalg



In [ ]:
si_Label = StringIndexer(inputCol="BAD_YIELD", outputCol="label").fit(spark_df)
label_converter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=si_Label.labels)

In [ ]:
pd_data.columns

In [ ]:
va_features = VectorAssembler(inputCols=['WS_001_FLOW_MEAN', 'WS_001_FLOW_MIN', 'WS_001_FLOW_MAX',
       'WS_001_CONC_MEAN', 'WS_001_CONC_MIN', 'WS_001_CONC_MAX',
       'DMW_FLOW_MEAN', 'DMW_FLOW_MIN', 'DMW_FLOW_MAX', 'ALK_FLOW_MEAN',
       'ALK_FLOW_MIN', 'ALK_FLOW_MAX', 'RPM_MEAN', 'RPM_MIN', 'RPM_MAX', 'PLANT_A'], outputCol="features")

In [ ]:
from pyspark.ml.classification import RandomForestClassifier
classifier = RandomForestClassifier(featuresCol="features")

pipeline = Pipeline(stages=[ si_Label, va_features, classifier, label_converter])
model = pipeline.fit(train_data)

In [ ]:
predictions = model.transform(test_data)
evaluatorDT = BinaryClassificationEvaluator(rawPredictionCol="prediction")
area_under_curve = evaluatorDT.evaluate(predictions)

#default evaluation is areaUnderROC
print("areaUnderROC = %g" % area_under_curve)

# Save and deploy the model

from scipy import sparse
from scipy import linalg

In [ ]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient 
import json 



wml_client = WatsonMachineLearningAPIClient(WML_CREDENTIALS)

### Remove existing model and deployment

In [ ]:
model_deployment_ids = wml_client.deployments.get_uids()
for deployment_id in model_deployment_ids:
    deployment = wml_client.deployments.get_details(deployment_id)
    model_id = deployment['entity']['deployable_asset']['guid']
    if deployment['entity']['name'] == DEPLOYMENT_NAME:
        print('Deleting deployment id', deployment_id)
        wml_client.deployments.delete(deployment_id)
        print('Deleting model id', model_id)
        wml_client.repository.delete(model_id)
wml_client.repository.list_models()

In [ ]:
model_props = {
    wml_client.repository.ModelMetaNames.NAME: "{}".format(MODEL_NAME),
    wml_client.repository.ModelMetaNames.EVALUATION_METHOD: "binary",
    wml_client.repository.ModelMetaNames.EVALUATION_METRICS: [
        {
           "name": "areaUnderROC",
           "value": area_under_curve,
           "threshold": 0.85
        }
    ]
}

In [ ]:
wml_models = wml_client.repository.get_details()
model_uid = None
for model_in in wml_models['models']['resources']:
    if MODEL_NAME == model_in['entity']['name']:
        model_uid = model_in['metadata']['guid']
        break

if model_uid is None:
    print("Storing model ...")

    published_model_details = wml_client.repository.store_model(model=model, meta_props=model_props, training_data=train_data, pipeline=pipeline)
    model_uid = wml_client.repository.get_model_uid(published_model_details)
    print("Done")

In [ ]:
model_uid

In [ ]:
wml_deployments = wml_client.deployments.get_details()
deployment_uid = None
for deployment in wml_deployments['resources']:
    if DEPLOYMENT_NAME == deployment['entity']['name']:
        deployment_uid = deployment['metadata']['guid']
        break

if deployment_uid is None:
    print("Deploying model...")

    deployment = wml_client.deployments.create(artifact_uid=model_uid, name=DEPLOYMENT_NAME, asynchronous=False)
    deployment_uid = wml_client.deployments.get_uid(deployment)
    
print("Model id: {}".format(model_uid))
print("Deployment id: {}".format(deployment_uid))

# Configure OpenScale

In [ ]:
from ibm_ai_openscale import APIClient
from ibm_ai_openscale.engines import *
from ibm_ai_openscale.utils import *
from ibm_ai_openscale.supporting_classes import PayloadRecord, Feature
from ibm_ai_openscale.supporting_classes.enums import *

### Get AI OpenScale GUID

In [ ]:
import requests

AIOS_GUID = None
token_data = {
    'grant_type': 'urn:ibm:params:oauth:grant-type:apikey',
    'response_type': 'cloud_iam',
    'apikey': CLOUD_API_KEY
}

response = requests.post('https://iam.bluemix.net/identity/token', data=token_data)
iam_token = response.json()['access_token']
iam_headers = {
    'Content-Type': 'application/json',
    'Authorization': 'Bearer %s' % iam_token
}

resources = json.loads(requests.get('https://resource-controller.cloud.ibm.com/v2/resource_instances', headers=iam_headers).text)['resources']
for resource in resources:
    if "aiopenscale" in resource['id'].lower():
        AIOS_GUID = resource['guid']
        
AIOS_CREDENTIALS = {
    "instance_guid": AIOS_GUID,
    "apikey": CLOUD_API_KEY,
    "url": "https://api.aiopenscale.cloud.ibm.com"
}

if AIOS_GUID is None:
    print('AI OpenScale GUID NOT FOUND')
else:
    print(AIOS_GUID)

## Create schema and datamart

In [ ]:
ai_client = APIClient(aios_credentials=AIOS_CREDENTIALS)
ai_client.version
time.sleep(20)

### Set up datamart

In [ ]:
try:
    data_mart_details = ai_client.data_mart.get_details()
    if 'internal_database' in data_mart_details and data_mart_details['internal_database']:
        if KEEP_MY_INTERNAL_POSTGRES:
            print('Using existing internal datamart.')
        else:
            if DB_CREDENTIALS is None:
                print('No postgres credentials supplied. Using existing internal datamart')
            else:
                print('Switching to external datamart')
                ai_client.data_mart.delete(force=True)
                ai_client.data_mart.setup(db_credentials=DB_CREDENTIALS)
    else:
        print('Using existing external datamart')
except:
    if DB_CREDENTIALS is None:
        print('Setting up internal datamart')
        ai_client.data_mart.setup(internal_db=True)
    else:
        print('Setting up external datamart')
        ai_client.data_mart.setup(db_credentials=DB_CREDENTIALS)
    

In [ ]:
data_mart_details = ai_client.data_mart.get_details()
data_mart_details

## Bind machine learning engines

In [ ]:
binding_uid = ai_client.data_mart.bindings.add('WML instance', WatsonMachineLearningInstance(WML_CREDENTIALS))
if binding_uid is None:
    binding_uid = ai_client.data_mart.bindings.get_details()['service_bindings'][0]['metadata']['guid']
bindings_details = ai_client.data_mart.bindings.get_details()
ai_client.data_mart.bindings.list()

In [ ]:
print(binding_uid)

In [ ]:
ai_client.data_mart.bindings.list_assets()

## Subscriptions

### Remove existing  propensity to buy subscriptions

In [ ]:
subscriptions_uids = ai_client.data_mart.subscriptions.get_uids()
for subscription in subscriptions_uids:
    sub_name = ai_client.data_mart.subscriptions.get_details(subscription)['entity']['asset']['name']
    if sub_name == MODEL_NAME:
        ai_client.data_mart.subscriptions.delete(subscription)
        print('Deleted existing subscription for', MODEL_NAME)

In [ ]:
subscription = ai_client.data_mart.subscriptions.add(WatsonMachineLearningAsset(
    model_uid,
    problem_type=ProblemType.BINARY_CLASSIFICATION,
    input_data_type=InputDataType.STRUCTURED,
    label_column='BAD_YIELD',
    prediction_column='predictedLabel',
    probability_column='probability',
    feature_columns = ['WS_001_FLOW_MEAN', 'WS_001_FLOW_MIN', 'WS_001_FLOW_MAX',
       'WS_001_CONC_MEAN', 'WS_001_CONC_MIN', 'WS_001_CONC_MAX',
       'DMW_FLOW_MEAN', 'DMW_FLOW_MIN', 'DMW_FLOW_MAX', 'ALK_FLOW_MEAN',
       'ALK_FLOW_MIN', 'ALK_FLOW_MAX', 'RPM_MEAN', 'RPM_MIN', 'RPM_MAX','PLANT_A'],
    categorical_columns = ['PLANT_A']
))

if subscription is None:
    print('Subscription already exists; get the existing one')
    subscriptions_uids = ai_client.data_mart.subscriptions.get_uids()
    for sub in subscriptions_uids:
        if ai_client.data_mart.subscriptions.get_details(sub)['entity']['asset']['name'] == MODEL_NAME:
            subscription = ai_client.data_mart.subscriptions.get(sub)

Get subscription list

In [ ]:
subscriptions_uids = ai_client.data_mart.subscriptions.get_uids()
ai_client.data_mart.subscriptions.list()

In [ ]:

subscription.get_details()

### Score the model so we can configure monitors

In [ ]:
propensity_to_buy_scoring_endpoint = None
print(deployment_uid)

for deployment in wml_client.deployments.get_details()['resources']:
    if deployment_uid in deployment['metadata']['guid']:
        propensity_to_buy_scoring_endpoint = deployment['entity']['scoring_url']
        
print(propensity_to_buy_scoring_endpoint)

In [ ]:
fields = ['WS_001_FLOW_MEAN', 'WS_001_FLOW_MIN', 'WS_001_FLOW_MAX',
       'WS_001_CONC_MEAN', 'WS_001_CONC_MIN', 'WS_001_CONC_MAX',
       'DMW_FLOW_MEAN', 'DMW_FLOW_MIN', 'DMW_FLOW_MAX', 'ALK_FLOW_MEAN',
       'ALK_FLOW_MIN', 'ALK_FLOW_MAX', 'RPM_MEAN', 'RPM_MIN', 'RPM_MAX', 'PLANT_A']
values = [[91.472265,91.591823,91.400894,29.999306,29.9453,30.0347,88.597187,88.598325,88.595852,91.467674,91.52794,91.439629,5010.304375,5010.05,5010.61,1],
          [91.286568,91.333725,91.225539,29.999756,29.8993,30.1047,88.597187,88.598325,88.595852,91.467674,91.52794,91.439629,5011.043681,5010.74,5011.32,1],
          [91.581607,91.677855,91.57625,30.000624,29.9481,30.0648,88.597187,88.598325,88.595852,91.467674,91.52794,91.439629,5008.680625,5007.45,5009.42,0],
          [91.332992,91.419757,91.313216,30.000023,29.9221,30.1148,88.597187,88.598325,88.595852,91.467674,91.52794,91.439629,5010.7575,5010.47,5011.09,1],
          [87.178007,87.376229,87.19236,65.001133,64.8447,65.1378,92.93055,92.934958,92.91834,87.957784,88.105276,87.929657,5009.978056,5009.61,5010.3,1],
          [91.463103,91.591823,91.400894,29.999842,29.9367,30.0449,88.597187,88.598325,88.595852,91.467674,91.52794,91.439629,5010.263889,5009.75,5010.5,0],
          [87.284294,87.462261,87.280037,65.000094,64.9171,65.0708,92.93055,92.934958,92.91834,87.957784,88.105276,87.929657,5008.015347,5006.6,5009.73,1],
          [87.121198,87.290196,87.104682,65.000774,64.8889,65.1762,92.93055,92.934958,92.91834,86.453545,86.63842,86.425383,5009.210625,5008.67,5009.67,1],
          [87.2452,87.462261,87.19236,62.999506,62.8072,63.167,92.682929,92.68715,92.67134,87.957784,88.105276,87.929657,5011.408889,5011.12,5011.59,0],
          [87.099207,87.032099,87.280037,64.983894,62.8779,65.1535,92.93055,92.934958,92.91834,86.453545,86.63842,86.425383,5010.101181,5009.21,5010.83,0],
          [91.243197,91.333725,91.225539,29.998703,29.9305,30.1019,88.597187,88.598325,88.595852,91.467674,91.52794,91.439629,5010.714444,5010.44,5011.11,1],
          [91.612149,91.677855,91.57625,29.998469,29.9674,30.0269,88.597187,88.598325,88.595852,91.467674,91.52794,91.439629,5009.607431,5008.38,5010.55,1],
          [87.005748,87.204164,86.929326,65.001815,64.1377,65.9022,92.93055,92.934958,92.91834,86.453545,86.63842,86.425383,5010.824861,5010.53,5011.05,1],
          [91.260301,91.333725,91.225539,30.000992,29.9268,30.0831,88.597187,88.598325,88.595852,91.467674,91.52794,91.439629,5010.054444,5009.72,5010.31,1],
          [90.70443,90.731497,90.699472,29.513889,29.0,30.0,88.597187,88.598325,88.595852,90.464848,90.550036,90.43678,5007.784861,5006.61,5008.85,1],
          [91.538237,91.677855,91.488572,30.000409,29.9454,30.0518,88.597187,88.598325,88.595852,91.467674,91.52794,91.439629,5009.791319,5008.99,5010.11,1],
          [87.051561,87.204164,87.017004,64.999602,64.8108,65.146,92.93055,92.934958,92.91834,86.453545,86.63842,86.425383,5009.726389,5005.68,5010.44,0],
          [91.614593,91.763888,91.57625,29.999788,29.9509,30.0482,88.597187,88.598325,88.595852,91.467674,91.52794,91.439629,5008.214444,5006.21,5009.69,0],
          [91.365367,91.419757,91.313216,30.00108,29.8762,30.1167,88.597187,88.598325,88.595852,91.467674,91.52794,91.439629,4998.986875,4996.14,5003.11,0]
         ]
payload_scoring = {"fields": fields,"values": values}
scoring_response = wml_client.deployments.score(propensity_to_buy_scoring_endpoint, payload_scoring)

print(scoring_response)

## Quality and feedback monitoring

### Enable quality monitoring

Wait ten seconds to allow the payload logging table to be set up before we begin enabling monitors.

In [ ]:
time.sleep(20)
subscription.quality_monitoring.enable(threshold=0.7, min_records=100)

### Feedback logging

In [ ]:
!rm df_feedback.json
!wget https://raw.githubusercontent.com/shadgriffin/oglabworking/master/df_feedback.json

In [ ]:
with open('df_feedback.json') as feedback_file:
    df_feedback = json.load(feedback_file)
subscription.feedback_logging.store(df_feedback)

In [ ]:
subscription.feedback_logging.show_table()

In [ ]:
run_details = subscription.quality_monitoring.run()
status = run_details['status']
id = run_details['id']
print(id)

print("Run status: {}".format(status))

start_time = time.time()
elapsed_time = 0

while status != 'completed' and elapsed_time < 60:
    time.sleep(10)
    run_details = subscription.quality_monitoring.get_run_details(run_uid=id)
    status = run_details['status']
    elapsed_time = time.time() - start_time
    print("Run status: {}".format(status))

In [ ]:
subscription.quality_monitoring.get_run_details()

In [ ]:
subscription.quality_monitoring.show_table()

subscription.quality_monitoring._get_data_from_rest_api()

In [ ]:
ai_client.data_mart.get_deployment_metrics()

## Fairness monitoring

In [ ]:
subscription.fairness_monitoring.enable(
            features=[
                Feature("PLANT_A", majority=[[1,1]], minority=[[0,0]], threshold=0.95)
            ],
            favourable_classes=['GOOD'],
            unfavourable_classes=['BAD'],
            min_records=1000,
            training_data=df_training
        )

## Score the model again now that monitoring is configured

In [ ]:
!rm df_payload_biased.json
!wget https://raw.githubusercontent.com/shadgriffin/oglabworking/master/df_payload_biased.json

Score 1000 randomly chosen records

In [ ]:
import random

with open('df_payload_biased.json', 'r') as scoring_file:
    scoring_data = json.load(scoring_file)

fields = scoring_data['fields']
values = []
for _ in range(1000):
    values.append(random.choice(scoring_data['values']))
payload_scoring = {"fields": fields, "values": values}

scoring_response = wml_client.deployments.score(propensity_to_buy_scoring_endpoint, payload_scoring)
print(scoring_response)

In [ ]:
subscription.get_details()

# Insert historical payloads

In [ ]:
!rm payload_history*.json
!wget https://raw.githubusercontent.com/shadgriffin/propensitytobuylab/master/payload_history_1.json
!wget https://raw.githubusercontent.com/shadgriffin/propensitytobuylab/master/payload_history_2.json
!wget https://raw.githubusercontent.com/shadgriffin/propensitytobuylab/master/payload_history_3.json
!wget https://raw.githubusercontent.com/shadgriffin/propensitytobuylab/master/payload_history_4.json
!wget https://raw.githubusercontent.com/shadgriffin/propensitytobuylab/master/payload_history_5.json
!wget https://raw.githubusercontent.com/shadgriffin/propensitytobuylab/master/payload_history_6.json
!wget https://raw.githubusercontent.com/shadgriffin/propensitytobuylab/master/payload_history_7.json

In [ ]:
historyDays = 7
from ibm_ai_openscale.supporting_classes import PayloadRecord, Feature
import datetime
import time

In [ ]:
data_mart_id = subscription.get_details()['metadata']['url'].split('/service_bindings')[0].split('marts/')[1]
print(data_mart_id)

In [ ]:
performance_metrics_url = 'https://api.aiopenscale.cloud.ibm.com' + subscription.get_details()['metadata']['url'].split('/service_bindings')[0] + '/metrics'
print(performance_metrics_url)

## Insert historical fairness metrics

In [ ]:
!rm fairness_records.json
!wget https://raw.githubusercontent.com/shadgriffin/oglabworking/master/fairness_records.json
import random

In [ ]:

token_data = {
    'grant_type': 'urn:ibm:params:oauth:grant-type:apikey',
    'response_type': 'cloud_iam',
    'apikey': AIOS_CREDENTIALS['apikey']
}

response = requests.post('https://iam.bluemix.net/identity/token', data=token_data)
iam_token = response.json()['access_token']
iam_headers = {
    'Content-Type': 'application/json',
    'Authorization': 'Bearer %s' % iam_token
}

with open('fairness_records.json', 'r') as history_file:
    payloads = json.load(history_file)

for day in range(historyDays):
    print('Day', day + 1)
    for hour in range(24):
        score_time = (datetime.datetime.utcnow() + datetime.timedelta(hours=(-(24*day + hour + 1)))).strftime('%Y-%m-%dT%H:%M:%SZ')
        
        qualityMetric = {
            'metric_type': 'fairness',
            'binding_id': binding_uid,
            'timestamp': score_time,
            'subscription_id': model_uid,
            'asset_revision': model_uid,
            'deployment_id': deployment_uid,
            'value': random.choice(payloads)
        }

        response = requests.post(performance_metrics_url, json=[qualityMetric], headers=iam_headers)
print('Finished')

## Insert historical quality metrics

In [ ]:
token_data = {
    'grant_type': 'urn:ibm:params:oauth:grant-type:apikey',
    'response_type': 'cloud_iam',
    'apikey': AIOS_CREDENTIALS['apikey']
}

response = requests.post('https://iam.bluemix.net/identity/token', data=token_data)
iam_token = response.json()['access_token']
iam_headers = {
    'Content-Type': 'application/json',
    'Authorization': 'Bearer %s' % iam_token
}

measurements = [0.94, 0.91, 0.78, 0.82, 0.90, 0.94, 0.93]
for day in range(historyDays):
    print('Day', day + 1)
    for hour in range(24):
        score_time = (datetime.datetime.utcnow() + datetime.timedelta(hours=(-(24*day + hour + 1)))).strftime('%Y-%m-%dT%H:%M:%SZ')
        
        qualityMetric = {
            'metric_type': 'quality',
            'binding_id': binding_uid,
            'timestamp': score_time,
            'subscription_id': model_uid,
            'asset_revision': model_uid,
            'deployment_id': deployment_uid,
            'value': {
                'quality': measurements[day],
                'threshold': 0.85,
                'metrics': [
                    {
                        'name': 'auroc',
                        'value': measurements[day],
                        'threshold': 0.75
                    }
                ]
            }
        }

        response = requests.post(performance_metrics_url, json=[qualityMetric], headers=iam_headers)
print('Finished')

## Insert historical performance metrics

In [ ]:
token_data = {
    'grant_type': 'urn:ibm:params:oauth:grant-type:apikey',
    'response_type': 'cloud_iam',
    'apikey': AIOS_CREDENTIALS['apikey']
}

response = requests.post('https://iam.bluemix.net/identity/token', data=token_data)
iam_token = response.json()['access_token']
iam_headers = {
    'Content-Type': 'application/json',
    'Authorization': 'Bearer %s' % iam_token
}

for day in range(historyDays):
    print('Day', day + 1)
    for hour in range(24):
        score_time = (datetime.datetime.utcnow() + datetime.timedelta(hours=(-(24*day + hour + 1)))).strftime('%Y-%m-%dT%H:%M:%SZ')
        score_count = random.randint(600, 6000)
        score_resp = random.uniform(600, 3000)

        performanceMetric = {
            'metric_type': 'performance',
            'binding_id': binding_uid,
            'timestamp': score_time,
            'subscription_id': model_uid,
            'asset_revision': model_uid,
            'deployment_id': deployment_uid,
            'value': {
                'response_time': score_resp,
                'records': score_count
            }
        }

        response = requests.post(performance_metrics_url, json=[performanceMetric], headers=iam_headers)
print('Finished')

## Configure Explainability

In [ ]:
pd_data.head()

In [ ]:
from ibm_ai_openscale.supporting_classes import *
subscription.explainability.enable(training_data=df_training)

In [ ]:
subscription.explainability.get_details()

## Run fairness monitor

Kick off a fairness monitor run on current data. Depending on how fast the monitor runs, the table may not contain the most recent results.

In [ ]:
run_details = subscription.fairness_monitoring.run()

In [ ]:
subscription.fairness_monitoring.show_table()

## Additional data to help debugging

In [ ]:
#print('Datamart:', data_mart_id)
print('Model:', model_uid)
print('Deployment:', deployment_uid)
print('Binding:', binding_uid)
print('Scoring URL:', propensity_to_buy_scoring_endpoint)

## Identify transactions for Explainability

Transaction IDs identified by the cells below can be copied and pasted into the Explainability tab of the OpenScale dashboard.

In [ ]:
import json, random

DEPLOYMENT_NAME = "Yieldx Model"
MIN_RECORDS = 1000
MAX_RECORDS = 1000

In [ ]:
!rm df_payload_biased.json
!wget https://raw.githubusercontent.com/shadgriffin/oglabworking/master/df_payload_biased.json

In [ ]:
wml_deployments = wml_client.deployments.get_details()
scoring_url = None
for deployment in wml_deployments['resources']:
    if DEPLOYMENT_NAME == deployment['entity']['name']:
        scoring_url = deployment['entity']['scoring_url']
        break
    
print("Scoring URL: {}".format(scoring_url))

In [ ]:
try:
    with open('df_payload_biased.json', 'r') as scoring_file:
        scoring_data = json.load(scoring_file)
    print('file found')
    
except:
    !wget https://raw.githubusercontent.com/shadgriffin/oglabworking/master/df_payload_biased.json
    with open('df_payload_biased.json', 'r') as scoring_file:
        scoring_data = json.load(scoring_file)
    print('file downloaded')


In [ ]:
fields = scoring_data['fields']
values = []
for _ in range(0, random.randint(MIN_RECORDS, MAX_RECORDS)):
    values.append(random.choice(scoring_data['values']))
payload_scoring = {"fields": fields, "values": values}

scoring_response = wml_client.deployments.score(scoring_url, payload_scoring)
print(scoring_response)

In [ ]:
time.sleep(10)
payload_data = subscription.payload_logging.get_table_content(limit=200)
payload_data.filter(items=['scoring_id', 'predictedLabel', 'probability','PLANT_A'])

## Congratulations!

You have finished the hands-on lab for IBM Watson OpenScale. You can now view the [OpenScale Dashboard](https://aiopenscale.cloud.ibm.com/). Click on the tile for the Propensity to Buy model to see fairness, accuracy, and performance monitors. Click on the timeseries graph to get detailed information on transactions during a specific time window.

